rh_statement.csv is a robinhood report that can be downloaded from the account page.  Here we will parse the report and identify the options, stock, optione_events, deposits and withdrawals.

In [63]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('../output/rh_statement.csv', on_bad_lines='skip')

# Ensure 'Activity Date' is in datetime format
df['Activity Date'] = pd.to_datetime(df['Activity Date'], errors='coerce')

# Find the earliest Activity Date
earliest_activity_date = df['Activity Date'].min()

print(f'Earliest Activity Date: {earliest_activity_date}')

# Clean the 'Amount' column and convert to float
def clean_amount(row):
    amount_str = str(row['Amount']).replace('$', '').replace(',', '').strip()  # Clean the amount
    if amount_str == "":  # Check for empty string
        return 0.0 
    if '(' in amount_str:  # Check for debit
        amount_str = amount_str.replace('(', '').replace(')', '')  # Remove parentheses
        return -float(amount_str)  # Return negative for debits
    else:
        return float(amount_str)  # Return positive for credits

#Apply the function to clean the 'Amount' column
df['Amount'] = df.apply(clean_amount, axis=1)
current_balance = df['Amount'].sum() 
print(f'Current Balance: ${current_balance:.2f}') 

# Sum the amounts for rows with "ACH" in the 'Description' column
ach_rows = df[df['Description'].str.contains('ACH', na=False)]
ach_total = ach_rows['Amount'].sum()

# Sum the amounts for rows with "ACH" in the 'Description' column
option_rows = df[df['Description'].str.contains('Call|Put', na=False)]
option_total = option_rows['Amount'].sum()

# Sum the amounts for rows with "ACH" in the 'Description' column
option_event_rows = df[df['Description'].str.contains('Assigned|Exercised', na=False)]
print(f" option event count {len(option_event_rows)}")
option_event_total = option_event_rows['Amount'].sum()

stock_rows = df[df['Description'].str.contains('CUSIP', na=False)]
stock_rows = stock_rows[~stock_rows['Description'].str.contains('Options', na=False)]
stock_total = stock_rows['Amount'].sum()

sub_rows = df[df['Description'].str.contains('Subscription', na=False)]
sub_total = sub_rows['Amount'].sum()

interest_rows = df[df['Description'].str.contains('Interest', na=False)]
interest_total = interest_rows['Amount'].sum()

#Gold Deposit Boost Payment
boost_rows = df[df['Description'].str.contains('Gold Deposit Boost Payment', na=False)]
boost_total = boost_rows['Amount'].sum()

# print(f'Gold Boost Paid ${boost_total:.2f}')
print(f'Total Interest Paid ${interest_total:.2f}')
print(f'Total Gold Subscription cost ${sub_total:.2f}')
print(f'Total Stock cost ${stock_total:.2f}')
print(f'Total Options cost ${option_total:.2f}')
print(f'Total Option Events cost ${option_event_total:.2f}')
print(f'Total Equities cost ${stock_total + option_total + option_event_total:.2f}')

# Calculate total ACH deposits and withdrawals
ach_deposits = ach_rows[ach_rows['Amount'] > 0]['Amount'].sum()  # Total deposits
ach_withdrawals = ach_rows[ach_rows['Amount'] < 0]['Amount'].sum()  # Total withdrawals

print(f'Total ACH Deposits: ${ach_deposits:.2f}')
print(f'Total ACH Withdrawals: ${ach_withdrawals:.2f}')
print(f'Total ACH transactions: ${ach_total:.2f}')

# Filter for rows with 'Activity Date' in the year 2024
# df_2024 = ach_rows[ach_rows['Activity Date'].dt.year == 2024]
# total_2024_amount = df_2024['Amount'].sum()
# #print(df_2024['Amount'])
# print(f'Total Amount for the year 2024: ${total_2024_amount:.2f}')

# Save the resulting DataFrame to a new CSV file
#ach_rows.to_csv('../output/ach_transactions.csv', index=False)

Earliest Activity Date: 2018-11-12 00:00:00
Current Balance: $564.98
 option event count 13
Total Interest Paid $42.42
Total Gold Subscription cost $-65.00
Total Stock cost $3426.80
Total Options cost $-2089.24
Total Option Events cost $-9115.66
Total Equities cost $-7778.10
Total ACH Deposits: $19995.61
Total ACH Withdrawals: $-14998.61
Total ACH transactions: $4997.00
